In [1]:
import os
import pandas as pd
import csv
import regex as re
import xlwt
import glob
import statistics
from datetime import datetime
from statistics import mode
import collections
import itertools
from iteration_utilities import deepflatten
import csv
from collections import Counter 
import timeit


numbers = r"""(?x)          # Turn on free spacing mode
            (
              #^a(?=\s)|     # Here we match a at the start of string before  whitespace
              #[-]?[0-9]+[,.]?[0-9]*[\/][0-9]+[,.]?[0-9]*|  # new numbers
              (?<!-\d*\.*|\.|table\s\d*\.*\d*\.*\d*\.*)\b[0-9]+[,.]?[0-9]*|  # new numbers
              (?<!-\d*\.*|\.|table\s\d*\.*\d*\.*\d*\.*)\b\d*\.?\,?\\?\d+ # HERE we match one or more digits
              #\b            # Initial word boundary 
              #(?:
              #    one|two|three|four|five|six|seven|eight|nine|ten| 
              #    eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen| 
              #    eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty| 
              #    ninety|hundred|thousand|half
              #)             # A list of alternatives
              #\b            # Trailing word boundary
              )"""   

def fnote_fix(string):
    nums = re.findall(numbers, string, flags=re.IGNORECASE)

    nums_pr = [n for n in nums if "," in n]

    for n, num in enumerate(nums_pr):
        newnum = num.split(",")
        for i, j in enumerate(newnum):
            if len(j) > 3:
                newnum[i] = j[:3]
                finnum = "".join(newnum)
                string = re.sub(str(num), str(finnum), string)

    return string

def pre_processing(lines):
    lines1 = re.sub(r'"\d+', '', lines)
    lines2 = re.sub(r'[^\x00-\x7f]', r'-', lines1)
    lines3 = ' '.join(lines2.split())
    lines4 = re.sub(r'\n', ' ', lines3)  # takes away all new line indicators and replaces with a space, fixed a problem where regex wasn't catching some words when ran in the loop
    lines5 = re.sub(r'\t', ' ', lines4)  # removes \t characters
    lines6 = re.sub(r'\\', '', lines5)
    lines7 = re.sub(r'http\S+', '', lines6)
    lines8 = re.sub(r'\d+\.\d{3,}\.*\d*\.*\d*|\d+\.\d+\.\d+|\+\-+|\-{3,}', '', lines7)
    lines9 = re.sub(r'\[\d+\]', '', lines8)
    lines10 = re.sub(r'=', ' ', lines9)
    lines11 = re.sub(r'\bsf\b', "square feet", lines10)
    lines12 = re.sub(
        r'\b(Sterling\sCodifiers\,\sInc\.|Article|Chapter|Section|SECTION|Sections|Subsection|Sec|Prior\scode|Code|Ordinance|Ord|Lots|through|pg|pgs|Part)\b\.*\s*(No\.)*\s*\d*\w*\:*\.*\-*\d*\-*\d*\,*\.*\s*(and)*\s*\d*\-*\d*\-*\d*\,*\s*(and)*\s*\d*\-*\d*\-*\d*',
        '', lines11)
    lines13 = re.sub(r'\bAmended\sby\sOrd\.\sNo\.\s\d+\,\s\d+\/d+\/\d+|Amended\s\d{4}\b|amended\s\d{1,2}\/\d{1,2}\/\d{1,4}|\d+\/\d+\/\d+', '', lines12)
    lines14 = re.sub(r'\d+\:\d+', '', lines13)
    lines15 = lines14.lower()
    lines16 = re.sub(r'\b(january|february|march|april|may|june|july|august|september|october|november|december)\b\s\d+\,*\s\d{4}', '', lines15)
    lines17 = re.sub(r'pg\.|pgs\.\s\d+\-\d+|page\s\d+', '', lines16)
    lines18 = re.sub(r'\bft\.|\bft\b', "feet ", lines17)
    lines20 = fnote_fix(str(lines18))
    lines21 = re.sub(r'[",!?*\[\]]', '', lines20)
    lines22 = re.sub(r';', ' ', lines21)
    lines23 = re.sub(r'\/acre|unit\/acre|unit\/net\sacre|unit\/gross\sacre|du\/ac\b', "unit per acre", lines22)
    lines24 = re.sub(r'\d+\/\d+\/\d+\ssterling\scodifiers\sinc\.', '', lines23)
    text = lines24.replace("(", "").replace(")", "").replace(".", "").replace(":", "").replace("/", "")
    text = text.replace("-", "").replace("'", "").replace(",", "").replace('``', '').replace("''", "")
    text = text.replace("[", "").replace("]", "")
    return text



def write_to_csv(outputfilename, header, data):
    # Create and write to the CSV file
    with open(outputfilename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write the header
        for entry in data:
            writer.writerow([entry])
    print(f"CSV file created at: {outputfilename}")

def extract_filename(path):
    import os
    base_name = os.path.basename(path)
    return os.path.splitext(base_name)[0]
def create_file(filedirect,outputfilename):
    filenames = sorted(glob.glob(filedirect + "*.txt")) # finds all .txt files in the folder your code is saved in
    row_filenames = [extract_filename(path) for path in filenames]
    header = ["Location", "Total_word_count"]
    extracted_filenames = [extract_filename(path) for path in filenames]
    write_to_csv(outputfilename, header, extracted_filenames)


def add_to_csv(outputfilename, counter, entry):
    # Read the existing CSV file into a DataFrame
    df = pd.read_csv(outputfilename, encoding='ISO-8859-1')
    
    # Modify the specific cell (counter row, second column)
    df.iloc[counter, 1] = entry
    
    # Save the DataFrame back to the CSV file
    df.to_csv(outputfilename, index=False, encoding='ISO-8859-1')


def main(filenames, outputfilename,filedirect):
    create_file(filedirect,outputfilename)
    for counter, loc in enumerate(filenames):
        with open(loc, 'r', errors='replace') as file:
            lines = file.read()
            text = pre_processing(lines)
            pre_processing_time = timeit.timeit(lambda: pre_processing(text), number=10)
            print(f"Execution time for pre_processing (1000 iterations): {pre_processing_time} seconds")

            total_word_count= len(text.split())
            split_time = timeit.timeit(lambda: len(text.split()), number=10)
            print(f"Execution time for len(text.split()) (1000 iterations): {split_time} seconds")

            add_to_csv(outputfilename, counter, entry=total_word_count)
            add_to_csv_time = timeit.timeit(lambda: add_to_csv(outputfilename, counter, entry=0), number=10)
            print(f"Execution time for add_to_csv (1000 iterations): {add_to_csv_time} seconds")


#id=0
id = int(os.getenv("SGE_TASK_ID", 1))
outputfilename = f"Total_word_count_{id}.csv"
filedirect = f"/restricted/projectnb/trucks/William/Mleczko_and_Desmond/nzlud/municipal_codes_all/{id}/"
#f"C:/Users/clint/Desktop/nzlud/municipal_codes_all/set{id}/"
filenames = sorted(glob.glob(filedirect + "*.txt"))
#number_of_folders=1
main(filenames, outputfilename,filedirect)



In [2]:


# for id in range(0, number_of_folders):
#     outputfilename = f"Total_word_count_{id}.csv"
#     filedirect = f"C:/Users/clint/Desktop/nzlud/municipal_codes_all/set{id}/"
#     filenames = sorted(glob.glob(filedirect + "*.txt"))
#     main(filenames, outputfilename,filedirect)

CSV file created at: Total_word_count_0.csv
Execution time for pre_processing (1000 iterations): 27.96431389998179 seconds
Execution time for len(text.split()) (1000 iterations): 0.27912299998570234 seconds
Execution time for add_to_csv (1000 iterations): 0.029060399974696338 seconds
Execution time for pre_processing (1000 iterations): 38.52287950000027 seconds
Execution time for len(text.split()) (1000 iterations): 0.3788734000409022 seconds
Execution time for add_to_csv (1000 iterations): 0.025686100008897483 seconds
